## Final Project Submission

Please fill out:
* Student name: Gabriel Araujo
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: James Irving 
* Blog post URL:


In [ ]:
#import pandas as pd
#import seaborn as sns
#from bs4 import BeautifulSoup
#import os
#import glob


In [ ]:
#file_names = os.listdir('zippedData')
#file_names

In [ ]:
#file_path = glob.glob('zippedData/*')
#file_path[0].split('\\')[-1]

In [ ]:
#data_dict = {}
#for file in file_path:
    #key = file.split('\\')[-1]
    #try:
        #df = pd.read_csv(file)
    #except:
        #df = pd.read_csv(file,delimiter='\t',encoding= 'unicode_escape' )
    #data_dict[key] = df

In [ ]:
#data_dict[file_names[-2]]

In [ ]:
#imbd_dataset = {}
#rt_dataset = {}
#bom_dataset = {}
#tn_dataset = {}
#tmdb_dataset = {}
#for key,value in data_dict.items():
    #if key.split('.')[0] == 'imdb':
        #imbd_dataset[key] = value
    #elif key.split('.')[0] == 'rt':
        #rt_dataset[key] = value
    #elif key.split('.')[0] == 'bom':
        #bom_dataset[key] = value
    #elif key.split('.')[0] == 'tn':
        #tn_dataset[key] = value
    #elif key.split('.')[0] == 'tmdb':
        #tmdb_dataset[key] = value

In [ ]:
#for key,value in imbd_dataset.items():
    #print(value.columns)

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

  # TRIAL CODE

In [12]:
#PAGE TO GET DATA FROM
html_page = requests.get('https://www.themoviedb.org/movie/top-rated?page=9')
soup = BeautifulSoup(html_page.content, 'html.parser')

In [13]:
# GETTING SPECIFIC MOVIE URL TO EXTRACT THE DATA AND ADDING IT TO A LIST
base_url = 'https://www.themoviedb.org/'
movies_data = soup.find('section', id='media_results')
movie_entries = movies_data.findAll('a', class_='image')
movie_titles = []
movie_pages = []

#LINK CONSISTS OF BASE_URL LINK + MODIFIED MOVIE TITLE
for entry in movie_entries:    
    title = entry.get('title')
    ext_1 = entry.get('href').split('/movie/')
    ext_2 = title.replace(' ','-')
    url = base_url+'movie/'+ext_1[1]+'-'+ext_2
    movie_titles.append(title)
    movie_pages.append(url)

In [14]:
movie_titles

['Nights of Cabiria',
 'The Circus',
 'My Neighbor Totoro',
 'Room',
 'Central Station',
 'The General',
 'Incendies',
 'On the Waterfront',
 'Alien',
 'Django Unchained',
 'Eternal Sunshine of the Spotless Mind',
 'The Lives of Others',
 'Das Boot',
 'The Elephant Man',
 'The Way He Looks',
 'What Ever Happened to Baby Jane?',
 'La Haine',
 'The Sting',
 'The Truman Show',
 'Memories of Murder']

In [ ]:
#TRIAL TO GET DATA FROM SPECIFIC PAGE
html_page_2 = requests.get(movie_pages[4])
soup_2 = BeautifulSoup(html_page_2.content, 'html.parser')

#GETTING CAST INFORMATION
html_page_3 = requests.get(movie_pages[4]+'/cast')
soup_3 = BeautifulSoup(html_page_3.content, 'html.parser')

In [60]:
#for i in list(range(0,20)):
    html_page_2 = requests.get(movie_pages[15])
    soup_2 = BeautifulSoup(html_page_2.content, 'html.parser')
    html_page_3 = requests.get(movie_pages[15]+'/cast')
    soup_3 = BeautifulSoup(html_page_3.content, 'html.parser')
    
    #GETTING MOVIE TITLE
    specific_movie_data_1 = soup_2.find('div', class_='single_column')
    title = specific_movie_data_1.find('h2').text.split('(')[0].strip()
    print(title)
    
    #GETTING RELEASE DATE
    release_date = specific_movie_data_1.find('span',class_='release').text.strip().split('(')[0]
    print(release_date)
    
    #GETTING GENRE LIST
    genre_list = []
    for genre in specific_movie_data_1.find('span',class_='genres').findAll('a'):
        genre_list.append(genre.text)
    print(genre_list)
    
    #GETTING LENGTH OF MOVIE
    run_time = specific_movie_data_1.find('span',class_='runtime').text.strip()
    print(run_time)
    
    #GETTING AVERAGE MOVIE RATING
    user_score = specific_movie_data_1.find('div',class_='percent').find('span').get('class')[1].split('-r')[-1]
    print(user_score)
    
    #GETTING LANGUAGE, BUDGET, AND REVENUE FROM FACTS LEFT_COLUMN
    specific_movie_data_2 = soup_2.find('section', class_='facts left_column')
    for data in specific_movie_data_2.findAll('p'):
        if data.text.split()[1] == 'Language':
            original_language = data.text.split()[-1]
            print(original_language)
            
        #USED NESTED IF STATEMENTS BECAUSE SOME MOVIES DON'T HAVE BUDGET OR REVENUE DATA
        elif data.text.split()[0] == 'Budget':
            if data.text.split()[-1] == '-':
                movie_budget = None
                print(movie_budget)
            else:
                movie_budget = int(data.text.split()[-1].strip('$').replace(',','').replace('.00',''))
                print(movie_budget)
        elif data.text.split()[0] == 'Revenue':
            if data.text.split()[-1] == '-':
                movie_revenue = None
                print(movie_revenue)
            else:        
                movie_revenue = int(data.text.split()[-1].strip('$').replace(',','').replace('.00',''))
                print(movie_revenue)
                
    #GETTING MOVIE KEYWORDS
    specific_movie_data_3 = soup_2.find('section',class_='keywords right_column')
    keyword_list = []
    for keyword in specific_movie_data_3.findAll('a'):
        keyword_list.append(keyword.text)
    print(keyword_list)
    
    #GETTING LIST OF ACTORS
    specific_movie_data_4 = soup_2.find('ol',class_='people scroller')
    actors_list = []
    #USED TRY BECAUSE SOME PAGES ARE MISSING ACTORS DATA
    try:
        for actor in specific_movie_data_4.findAll('p'):
            if actor.find('a') != None:
                if actor.find('a').text != 'View More ':
                    actors_list.append(actor.find('a').text)
        print(actors_list)
    except:
        actors_list.append(None)
        print(actors_list)
        
     #GETTING DIRECTOR'S NAME
    try:
        specific_movie_data_5 = soup_3.findAll('section',class_='panel pad')[1]
        #print(specific_movie_data_5)
        count = 0
        crew_list = []
       
        for crew_member in specific_movie_data_5.findAll('p'):
            count += 1
            crew_list.append(crew_member)
            if crew_member.text.strip() == 'Director':
                director_position = count-2
                director = crew_list[director_position].text
                print(director)
                print('\n')
    except:
        director = None
        print(director)
#writer = crew_list[writer_position].text
#print(writer)
#visual_prod = crew_list[visual_position].text
#print(visual_prod)

What Ever Happened to Baby Jane?
10/31/1962 
['Drama', 'Horror', 'Thriller']
2h 15m
81
English
980000
None
['sibling relationship', 'based on novel or book', 'aging', 'insanity', 'murder', 'hollywood', 'mental illness', 'invalid', 'vaudeville', 'recluse', 'actress', 'old mansion', 'former film star', 'hagsploitation']
['Bette Davis', 'Joan Crawford', 'Victor Buono', 'Wesley Addy', 'Julie Allred', 'Anne Barton', 'Marjorie Bennett', 'Bert Freed']
None


# WEBSCRAPPING FUNCTIONS

In [7]:
#FUNCTION TO RETRIEVE URL FOR THE NEXT PAGE
def next_page(base_url):
    next_page_num = int(base_url.split('=')[-1]) +1
    base_url = base_url.split('=')[0] + '='+ str(next_page_num)
    return(base_url)

In [8]:
#TESTING FUNCTION
next_page('https://www.themoviedb.org/movie/top-rated?page=1')

'https://www.themoviedb.org/movie/top-rated?page=2'

In [64]:
#Function to Scrape all of the Movies on the Given Movies Page
def scrape_movies_list(base_url):
    html_page = requests.get(base_url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    base_movie_url = 'https://www.themoviedb.org/'
    movies_data = soup.find('section', id='media_results')
    movie_entries = movies_data.findAll('a', class_='image')
    movie_titles = []
    movie_pages = []
    df_list = []
    for entry in movie_entries:    
        title = entry.get('title')
        ext_1 = entry.get('href').split('/movie/')
        ext_2 = title.replace(' ','-')
        url = base_movie_url+'movie/'+ext_1[1]+'-'+ext_2
        movie_titles.append(title)
        movie_pages.append(url)
    for movie in movie_pages:
        time.sleep(.2)
        html_page_2 = requests.get(movie)
        soup_2 = BeautifulSoup(html_page_2.content, 'html.parser')
        html_page_3 = requests.get(movie+'/cast')
        soup_3 = BeautifulSoup(html_page_3.content, 'html.parser')
        specific_movie_data_1 = soup_2.find('div', class_='single_column')
        title = specific_movie_data_1.find('h2').text.split('(')[0].strip()
        
        release_date = specific_movie_data_1.find('span',class_='release').text.strip().split('(')[0]
        
        genre_list = []
        for genre in specific_movie_data_1.find('span',class_='genres').findAll('a'):
            genre_list.append(genre.text)
        
        run_time = specific_movie_data_1.find('span',class_='runtime').text.strip()
        
        user_score = specific_movie_data_1.find('div',class_='percent').find('span').get('class')[1].split('-r')[-1]
        
        specific_movie_data_2 = soup_2.find('section', class_='facts left_column')
        for data in specific_movie_data_2.findAll('p'):
            if data.text.split()[1] == 'Language':
                original_language = data.text.split()[-1]
                
            elif data.text.split()[0] == 'Budget':
                if data.text.split()[-1] == '-':
                    movie_budget = None
                    
                else:
                    movie_budget = int(data.text.split()[-1].strip('$').replace(',','').replace('.00',''))
                    
            elif data.text.split()[0] == 'Revenue':
                if data.text.split()[-1] == '-':
                    movie_revenue = None
                    
                else:        
                    movie_revenue = int(data.text.split()[-1].strip('$').replace(',','').replace('.00',''))
                    
        specific_movie_data_3 = soup_2.find('section',class_='keywords right_column')
        keyword_list = []
        for keyword in specific_movie_data_3.findAll('a'):
            keyword_list.append(keyword.text)
        
        specific_movie_data_4 = soup_2.find('ol',class_='people scroller')
        actors_list = []
        try:
            for actor in specific_movie_data_4.findAll('p'):
                if actor.find('a') != None:
                    if actor.find('a').text != 'View More ':
                        actors_list.append(actor.find('a').text)
        except:
            actors_list.append(None)
        try:
            specific_movie_data_5 = soup_3.findAll('section','panel pad')[1]
            count = 0
            crew_list = []
            for crew_member in specific_movie_data_5.findAll('p'):
                count += 1
                crew_list.append(crew_member)
                if crew_member.text.strip() == 'Director':
                    director_position = count-2
                    director = crew_list[director_position].text
        except:
            director = None
        df = pd.DataFrame([title,release_date,genre_list,run_time,user_score,original_language,movie_budget,movie_revenue,keyword_list,actors_list,director]).transpose()
        df.columns = ['title','release_date','genre','run_time','user_score','original_language','movie_budget','movie_revenue','keywords','actors','director']
        df_list.append(df)
        df = pd.concat(df_list)
    return(df)

In [65]:
#TESTING SCRAPE_MOVIES_LIST FUNCTION
scrape_movies_list('https://www.themoviedb.org/movie/top-rated?page=9')

,title,release_date,genre,run_time,user_score,original_language,movie_budget,movie_revenue,keywords,actors,director
0,Nights of Cabiria,12/31/1957,[Drama],1h 50m,81,Italian,None,752045,"[rome italy, optimism, house, poverty, prostit...","[Giulietta Masina, François Périer, Franca Mar...",Federico Fellini
0,The Circus,01/06/1928,"[Comedy, Romance]",1h 12m,81,English,9000000,None,"[circus, black and white, silent film, employe...","[Charlie Chaplin, Al Ernest Garcia, Merna Kenn...",Charlie Chaplin
0,My Neighbor Totoro,04/16/1988,"[Fantasy, Animation, Family]",1h 26m,81,Japanese,3700000,45000000,"[sibling relationship, mother, leave, village,...","[Noriko Hidaka, Hitoshi Takagi, Chika Sakamoto...",Hayao Miyazaki
0,Room,11/06/2015,"[Drama, Thriller]",1h 57m,81,English,6000000,35401758,"[based on novel or book, carpet, isolation, ki...","[Brie Larson, Jacob Tremblay, Joan Allen, Will...",Lenny Abrahamson
0,Central Station,11/20/1998,[Drama],1h 53m,81,Portuguese,2900000,5596708,"[brazilian, rio de janeiro, letter, wilderness...","[Fernanda Montenegro, Vinícius de Oliveira, Ma...",Walter Salles
0,The General,01/15/1927,"[Action, Adventure, Comedy, Drama, War]",1h 19m,81,English,750000,None,"[spy, southern usa, general, fiancé, cannon, b...","[Buster Keaton, Marion Mack, Glen Cavender, Ji...",Buster Keaton
0,Incendies,09/04/2010,"[Drama, War, Mystery]",2h 10m,81,French,10000000,None,"[prison, middle east, rape, muslim, militia, i...","[Lubna Azabal, Mélissa Désormeaux-Poulin, Maxi...",Denis Villeneuve
0,On the Waterfront,07/28/1954,"[Crime, Drama]",1h 48m,81,English,910000,9600000,"[murder, union, dock, longshoreman, pigeon]","[Marlon Brando, Karl Malden, Lee J. Cobb, Rod ...",Elia Kazan
0,Alien,06/21/1979,"[Horror, Science Fiction]",1h 57m,81,English,11000000,104931801,"[android, spacecraft, countdown, space marine,...","[Tom Skerritt, Sigourney Weaver, Veronica Cart...",Ridley Scott
0,Django Unchained,12/25/2012,"[Drama, Western]",2h 45m,81,English,100000000,425368238,"[bounty hunter, hero, plantation, society, fri...","[Jamie Foxx, Christoph Waltz, Leonardo DiCapri...",Quentin Tarantino


In [66]:
#COMBINING NEXT_PAGE AND SCRAPE_MOVIES_LIST FUNCTIONS TO GATHER MOVIE INFO FROM WHOLE WEBSITE
i = 0 
lists_df = []
base_url = 'https://www.themoviedb.org/movie/top-rated?page=0'
while i <= 10:
    base_url = next_page(base_url)
    df = scrape_movies_list(base_url)
    i += 1
    lists_df.append(df)
    df = pd.concat(lists_df)
    print(i)
    time.sleep(.2)
df    

1
2
3
4
5
6
7
8
9
10
11


,title,release_date,genre,run_time,user_score,original_language,movie_budget,movie_revenue,keywords,actors,director
0,Dilwale Dulhania Le Jayenge,10/20/1995,"[Comedy, Drama, Romance]",3h 10m,88,Hindi,13200000,100000000,[],"[Shah Rukh Khan, Kajol, Amrish Puri, Anupam Kh...",Aditya Chopra
0,The Shawshank Redemption,10/14/1994,"[Drama, Crime]",2h 22m,87,English,25000000,28341469,"[prison, corruption, police brutality, prison ...","[Tim Robbins, Morgan Freeman, Bob Gunton, Clan...",Frank Darabont
0,The Godfather,03/15/1972,"[Drama, Crime]",2h 55m,87,English,6000000,245066411,"[italy, loss of loved one, love at first sight...","[Marlon Brando, Al Pacino, James Caan, Richard...",Francis Ford Coppola
0,Schindler's List,02/04/1994,"[Drama, History, War]",3h 15m,86,English,22000000,321365567,"[based on novel or book, factory, concentratio...","[Liam Neeson, Ben Kingsley, Ralph Fiennes, Car...",Steven Spielberg
0,Your Name.,08/26/2016,"[Romance, Animation, Drama]",1h 46m,86,Japanese,None,357986087,"[time travel, supernatural, afterlife, romance...","[Ryunosuke Kamiki, Mone Kamishiraishi, Masami ...",Makoto Shinkai
...,...,...,...,...,...,...,...,...,...,...,...
0,Hidden Figures,12/25/2016,"[Drama, History]",2h 7m,80,English,25000000,230698791,"[nasa, sexism, biography, mathematics, racial ...","[Taraji P. Henson, Octavia Spencer, Janelle Mo...",Theodore Melfi
0,Man with a Movie Camera,05/12/1929,[Documentary],1h 8m,80,Russian,None,None,"[soviet union, experimental, silent film, came...",[Mikhail Kaufman],Dziga Vertov
0,Amadeus,09/06/1984,"[Drama, History, Music]",2h 40m,80,English,18000000,51973029,"[italy, austria, composer, opera, talent, empe...","[F. Murray Abraham, Tom Hulce, Elizabeth Berri...",Miloš Forman
0,The Gold Rush,06/26/1925,"[Adventure, Comedy, Drama]",1h 35m,80,English,923000,2500000,"[river, gold, dance, worker, cabin, gold rush,...","[Charlie Chaplin, Georgia Hale, Mack Swain, To...",Charlie Chaplin
